In [ ]:
%load_ext autoreload
%autoreload 2

Please keep this notebook functionally at par with scripts/oncall/generate_report.py AND linearly runnable (no code with duplicated functionality; run all from this cell down should produce a report). Modify the parameters in `single_fitting_cycle` and `plot_trials` per customizations

In [ ]:
import numpy as np
import pandas as pd

import datetime
import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from main.seir.forecast import order_trials, get_all_trials
from utils.create_report import create_report, trials_to_df
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_trials


In [ ]:
t = time.time()
output_folder = f'../../reports/{t}'

## Params

In [ ]:
ktrials = 10

## Load Covid19india Data

In [ ]:
dataframes = get_dataframes_cached()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, train_period=21, val_period=7, num_evals=1000,
        data_from_tracker=False, initialisation='intermediate', model=SEIRHD, 
        smooth_jump=True,
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, train_period=21, val_period=0, num_evals=1000,
        data_from_tracker=False, initialisation='intermediate', model=SEIRHD, 
        smooth_jump=True,
        which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['smoothing_plot']

In [ ]:
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['df_loss']

## Create Master Loss Dataframe

### M1 Loss

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['m2']['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=False, error_bars=True)
    
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m1')
    predictions_dict[region]['m1']['params'] = params
    predictions_dict[region]['m1']['losses'] = losses
    predictions_dict[region]['m1']['predictions'] = predictions
    predictions_dict[region]['m1']['all_trials'] = trials_to_df(predictions, losses, params)
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m2')
    predictions_dict[region]['m2']['params'] = params
    predictions_dict[region]['m2']['losses'] = losses
    predictions_dict[region]['m2']['predictions'] = predictions
    predictions_dict[region]['m2']['all_trials'] = trials_to_df(predictions, losses, params)
    kforecasts = plot_trials(
        predictions_dict[region],
        train_fit='m2',
        predictions=predictions, 
        losses=losses, params=params, 
        k=ktrials,
        which_compartments=[Columns.confirmed, Columns.active])
    predictions_dict[region]['m2']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
    predictions_dict[region]['m2']['forecast_active_topk'] = kforecasts[Columns.active]


In [ ]:
predictions_dict[region]['m2']['forecast'].figure

In [ ]:
predictions_dict[region]['m2']['forecast_confirmed_topk'].figure

In [ ]:
predictions_dict[region]['m2']['forecast_active_topk'].figure

## Create Report


In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region], ROOT_DIR=output_folder)
    predictions_dict[region]['m1']['all_trials'].to_csv(os.path.join(output_folder, 'm1-trials.csv'))
    predictions_dict[region]['m2']['all_trials'].to_csv(os.path.join(output_folder, 'm2-trials.csv'))

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output, filename=os.path.join(output_folder, f'output-{t}.csv'))